# Creación de Splits (Patient-Level - SIN DATA LEAKAGE)

**Versión corregida:** Split por PACIENTES (no por casos)

**Problema anterior:**
- Split por casos → Mismo paciente en train Y val
- 90 pacientes con 35 consultas promedio c/u
- 100% leakage (modelo memoriza pacientes)

**Solución:**
- Split por Prontuario (patient ID)
- 72 pacientes train (80%) / 18 pacientes val (20%)
- 0% leakage (pacientes disjuntos)

**Estratificación:**
- Estratificar por CLASE MAYORITARIA del paciente
- Si paciente tiene 10 consultas ansiedad + 2 depresión → etiqueta = ansiedad
- Garantiza distribución similar en train/val

In [27]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from collections import Counter

# Configuración
DATA_PATH = Path('../data')
SPLITS_PATH = DATA_PATH / 'splits'
SPLITS_PATH.mkdir(exist_ok=True)

RANDOM_STATE = 42
TEST_SIZE = 0.2
PATIENT_COL = 'Prontuario'
LABEL_COL = 'Tipo'
TEXT_COL = 'Motivo Consulta'

print(f"Random state: {RANDOM_STATE}")
print(f"Test size: {TEST_SIZE}")

Random state: 42
Test size: 0.2


## 1. Cargar dataset

In [28]:
# Cargar ips_raw.csv
INPUT_FILE = DATA_PATH / 'ips_raw.csv'
df = pd.read_csv(INPUT_FILE)

print(f"Dataset cargado: {INPUT_FILE}")
print(f"Total casos: {len(df)}")
print(f"Columnas: {list(df.columns)}")
print(f"\nDistribución clases:")
print(df[LABEL_COL].value_counts())

Dataset cargado: ../data/ips_raw.csv
Total casos: 3155
Columnas: ['Archivo', 'Prontuario', 'Nombre Paciente', 'Sexo', 'Fecha Nacimiento', 'N° Consulta', 'Id', 'Fecha Consulta', 'Motivo Consulta', 'Tipo']

Distribución clases:
Tipo
depresion    2230
ansiedad      925
Name: count, dtype: int64


## 2. Análisis a nivel paciente

In [29]:
# Estadísticas por paciente
n_patients = df[PATIENT_COL].nunique()
n_cases = len(df)
avg_cases = n_cases / n_patients

patient_counts = df[PATIENT_COL].value_counts()

print("=" * 60)
print("ANÁLISIS A NIVEL PACIENTE")
print("=" * 60)
print(f"Pacientes únicos: {n_patients}")
print(f"Casos totales: {n_cases}")
print(f"Promedio casos/paciente: {avg_cases:.2f}")
print(f"Mediana casos/paciente: {patient_counts.median():.0f}")
print(f"Min-Max casos/paciente: {patient_counts.min()}-{patient_counts.max()}")

# Distribución
print(f"\nDistribución casos/paciente:")
print(patient_counts.describe())

ANÁLISIS A NIVEL PACIENTE
Pacientes únicos: 90
Casos totales: 3155
Promedio casos/paciente: 35.06
Mediana casos/paciente: 34
Min-Max casos/paciente: 6-65

Distribución casos/paciente:
count    90.000000
mean     35.055556
std      11.647791
min       6.000000
25%      27.000000
50%      33.500000
75%      42.750000
max      65.000000
Name: count, dtype: float64


## 3. Asignar clase mayoritaria por paciente

**Estrategia de estratificación:**
- Cada paciente tiene etiqueta = clase con más consultas
- Ejemplo: Paciente A con 8 consultas ansiedad + 2 depresión → etiqueta_paciente = 'ansiedad'
- Esto permite estratificar el split de pacientes

In [30]:
def get_patient_majority_label(patient_id, df, label_col):
    """Obtiene la clase mayoritaria de un paciente"""
    patient_labels = df[df[PATIENT_COL] == patient_id][label_col]
    most_common = Counter(patient_labels).most_common(1)[0][0]
    return most_common

# Crear DataFrame de pacientes con su clase mayoritaria
patients = df[PATIENT_COL].unique()
patient_labels = {p: get_patient_majority_label(p, df, LABEL_COL) for p in patients}

patients_df = pd.DataFrame({
    'Prontuario': list(patient_labels.keys()),
    'etiqueta_mayoritaria': list(patient_labels.values())
})

print("Distribución de pacientes por clase mayoritaria:")
print(patients_df['etiqueta_mayoritaria'].value_counts())
print(f"\nRatio ansiedad/depresión (pacientes): {patients_df['etiqueta_mayoritaria'].value_counts()['ansiedad'] / patients_df['etiqueta_mayoritaria'].value_counts()['depresion']:.2f}")

Distribución de pacientes por clase mayoritaria:
etiqueta_mayoritaria
depresion    57
ansiedad     33
Name: count, dtype: int64

Ratio ansiedad/depresión (pacientes): 0.58

etiqueta_mayoritaria
depresion    57
ansiedad     33
Name: count, dtype: int64

Ratio ansiedad/depresión (pacientes): 0.58


## 4. Split por PACIENTES (estratificado)

**CRÍTICO:** Dividir pacientes (no casos) para evitar data leakage

In [31]:
# Split de pacientes (80/20)
train_patients, val_patients = train_test_split(
    patients_df['Prontuario'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=patients_df['etiqueta_mayoritaria']  # Estratificar por clase mayoritaria
)

print("=" * 60)
print("SPLIT POR PACIENTES (PATIENT-LEVEL)")
print("=" * 60)
print(f"Pacientes en train: {len(train_patients)} ({len(train_patients)/n_patients*100:.1f}%)")
print(f"Pacientes en val: {len(val_patients)} ({len(val_patients)/n_patients*100:.1f}%)")

# Verificar no overlap
overlap = set(train_patients) & set(val_patients)
print(f"\n✓ Overlap: {len(overlap)} pacientes (DEBE SER 0)")
assert len(overlap) == 0, "ERROR: Hay pacientes en train Y val"

SPLIT POR PACIENTES (PATIENT-LEVEL)
Pacientes en train: 72 (80.0%)
Pacientes en val: 18 (20.0%)

✓ Overlap: 0 pacientes (DEBE SER 0)


## 5. Obtener índices de casos (train/val)

Ahora convertimos pacientes → casos (consultas)

In [32]:
# Filtrar casos por pacientes asignados
train_mask = df[PATIENT_COL].isin(train_patients)
val_mask = df[PATIENT_COL].isin(val_patients)

train_idx = df[train_mask].index.tolist()
val_idx = df[val_mask].index.tolist()

print("=" * 60)
print("CASOS RESULTANTES")
print("=" * 60)
print(f"Casos en train: {len(train_idx)} ({len(train_idx)/n_cases*100:.1f}%)")
print(f"Casos en val: {len(val_idx)} ({len(val_idx)/n_cases*100:.1f}%)")

# Distribución de clases en train/val
train_labels = df.loc[train_idx, LABEL_COL]
val_labels = df.loc[val_idx, LABEL_COL]

print(f"\nDistribución train:")
print(train_labels.value_counts())
print(f"Ratio A/D: {train_labels.value_counts()['ansiedad'] / train_labels.value_counts()['depresion']:.2f}")

print(f"\nDistribución val:")
print(val_labels.value_counts())
print(f"Ratio A/D: {val_labels.value_counts()['ansiedad'] / val_labels.value_counts()['depresion']:.2f}")

CASOS RESULTANTES
Casos en train: 2509 (79.5%)
Casos en val: 646 (20.5%)

Distribución train:
Tipo
depresion    1745
ansiedad      764
Name: count, dtype: int64
Ratio A/D: 0.44

Distribución val:
Tipo
depresion    485
ansiedad     161
Name: count, dtype: int64
Ratio A/D: 0.33


## 6. Verificación: No data leakage

In [33]:
# Verificar que no hay pacientes compartidos
train_patients_check = set(df.loc[train_idx, PATIENT_COL])
val_patients_check = set(df.loc[val_idx, PATIENT_COL])
overlap_check = train_patients_check & val_patients_check

print("=" * 60)
print("VERIFICACIÓN DATA LEAKAGE")
print("=" * 60)
print(f"Pacientes únicos en train: {len(train_patients_check)}")
print(f"Pacientes únicos en val: {len(val_patients_check)}")
print(f"Pacientes en AMBOS: {len(overlap_check)}")

if len(overlap_check) == 0:
    print("\n✅ VERIFICACIÓN EXITOSA: NO HAY DATA LEAKAGE")
else:
    print(f"\n❌ ERROR: {len(overlap_check)} pacientes en train Y val")
    print(f"Pacientes con leakage: {list(overlap_check)[:5]}")
    raise ValueError("DATA LEAKAGE DETECTADO")

VERIFICACIÓN DATA LEAKAGE
Pacientes únicos en train: 72
Pacientes únicos en val: 18
Pacientes en AMBOS: 0

✅ VERIFICACIÓN EXITOSA: NO HAY DATA LEAKAGE


## 7. Guardar splits

**IMPORTANTE:** Crear `row_id` único para cada caso (necesario para merge en baselines)

In [34]:
# Asignar row_id como índice original
df['row_id'] = df.index

# Preparar dataset base
df_base = df[['row_id', PATIENT_COL, TEXT_COL, LABEL_COL]].copy()
df_base.columns = ['row_id', 'patient_id', 'texto', 'etiqueta']

# Normalizar etiquetas
label_map = {'depresivo': 'depresion'}  # Mapeo si hay variantes
df_base['etiqueta'] = df_base['etiqueta'].str.lower().map(lambda x: label_map.get(x, x))

# Guardar archivos
dataset_base_path = SPLITS_PATH / 'dataset_base.csv'
train_indices_path = SPLITS_PATH / 'train_indices.csv'
val_indices_path = SPLITS_PATH / 'val_indices.csv'

df_base.to_csv(dataset_base_path, index=False, encoding='utf-8')
pd.DataFrame({'row_id': train_idx}).to_csv(train_indices_path, index=False)
pd.DataFrame({'row_id': val_idx}).to_csv(val_indices_path, index=False)

print("=" * 60)
print("ARCHIVOS GUARDADOS")
print("=" * 60)
print(f"Ubicación: {SPLITS_PATH}/")
print(f"\n1. dataset_base.csv")
print(f"   - {len(df_base)} casos")
print(f"   - Columnas: row_id, patient_id, texto, etiqueta")
print(f"\n2. train_indices.csv")
print(f"   - {len(train_idx)} índices (de {len(train_patients)} pacientes)")
print(f"\n3. val_indices.csv")
print(f"   - {len(val_idx)} índices (de {len(val_patients)} pacientes)")

ARCHIVOS GUARDADOS
Ubicación: ../data/splits/

1. dataset_base.csv
   - 3155 casos
   - Columnas: row_id, patient_id, texto, etiqueta

2. train_indices.csv
   - 2509 índices (de 72 pacientes)

3. val_indices.csv
   - 646 índices (de 18 pacientes)


## 8. Reporte final y comparación

Comparar distribución antes/después de split

In [35]:
print("\n" + "=" * 60)
print("RESUMEN: SPLIT POR PACIENTES (SIN LEAKAGE)")
print("=" * 60)

print(f"\n📊 DATASET COMPLETO:")
print(f"  - {n_patients} pacientes")
print(f"  - {n_cases} casos (consultas)")
print(f"  - {avg_cases:.1f} casos/paciente promedio")

print(f"\n✂️  SPLIT REALIZADO:")
print(f"  - Train: {len(train_patients)} pacientes ({len(train_idx)} casos)")
print(f"  - Val:   {len(val_patients)} pacientes ({len(val_idx)} casos)")
print(f"  - Ratio train/val: {len(train_patients)/len(val_patients):.1f}")

print(f"\n✅ VERIFICACIÓN:")
print(f"  - Pacientes compartidos: {len(overlap_check)} (0 = correcto)")
print(f"  - Estratificación por clase mayoritaria: ✓")
print(f"  - Random seed fijo: {RANDOM_STATE} (reproducible)")

print(f"\n⚠️  CAMBIO vs SPLIT ANTERIOR (por casos):")
print(f"  - Antes: 100% pacientes en train Y val (leakage total)")
print(f"  - Ahora: 0% pacientes compartidos (sin leakage)")
print(f"  - Impacto esperado: F1 score bajará 3-5% (métricas reales)")

print(f"\n📁 ARCHIVOS PARA BASELINES:")
print(f"  1. Cargar {dataset_base_path.name}")
print(f"  2. Cargar {train_indices_path.name} y {val_indices_path.name}")
print(f"  3. Filtrar usando row_id")
print(f"  4. ⚠️  RE-ENTRENAR TODOS LOS BASELINES con nuevos splits")

print("\n" + "=" * 60)
print("IMPORTANTE: Este split DEBE ser usado por TODOS los baselines")
print("para garantizar comparación justa y métricas válidas")
print("=" * 60)


RESUMEN: SPLIT POR PACIENTES (SIN LEAKAGE)

📊 DATASET COMPLETO:
  - 90 pacientes
  - 3155 casos (consultas)
  - 35.1 casos/paciente promedio

✂️  SPLIT REALIZADO:
  - Train: 72 pacientes (2509 casos)
  - Val:   18 pacientes (646 casos)
  - Ratio train/val: 4.0

✅ VERIFICACIÓN:
  - Pacientes compartidos: 0 (0 = correcto)
  - Estratificación por clase mayoritaria: ✓
  - Random seed fijo: 42 (reproducible)

⚠️  CAMBIO vs SPLIT ANTERIOR (por casos):
  - Antes: 100% pacientes en train Y val (leakage total)
  - Ahora: 0% pacientes compartidos (sin leakage)
  - Impacto esperado: F1 score bajará 3-5% (métricas reales)

📁 ARCHIVOS PARA BASELINES:
  1. Cargar dataset_base.csv
  2. Cargar train_indices.csv y val_indices.csv
  3. Filtrar usando row_id
  4. ⚠️  RE-ENTRENAR TODOS LOS BASELINES con nuevos splits

IMPORTANTE: Este split DEBE ser usado por TODOS los baselines
para garantizar comparación justa y métricas válidas
